# Write & Rewrite Barium forcing data in NEMO

In [1]:
import glob
import scipy.io
import numpy as np
import netCDF4 as nc
from __future__ import print_function
% matplotlib inline

#ORCA2

## River source

In [2]:
# Ba
Ba_name=glob.glob('_data/Exchange/Ba_ORCA2_SRC.mat')
Ba_Mat=scipy.io.loadmat(Ba_name[0])
Ba_ORCA2_2deg=Ba_Mat['Ba_ORCA2_2deg']
nav_lon=Ba_Mat['nav_lon']
nav_lat=Ba_Mat['nav_lat']
# d18O
d18O_name=glob.glob('_data/Exchange/d18O_ORCA2_SRC.mat')
d18O_Mat=scipy.io.loadmat(d18O_name[0])
d18O_ORCA2_2deg=d18O_Mat['d18O_ORCA2_2deg']

** Get `time_counter` from Xianmin's original file. **

In [3]:
ANHA4_name=glob.glob('_data/NEMO/ANHA4_runoff_monthly_DaiTrenberth_fewPTs.nc')
ANH4_obj=nc.Dataset(ANHA4_name[0])
time_counter=ANH4_obj.variables['time_counter'][:]

In [4]:
print(time_counter)

[  15.   46.   75.  106.  136.  167.  197.  228.  259.  289.  320.  350.]


In [5]:
nc_obj = nc.Dataset('_data/Exchange/TRC_ORCA2_SRC.nc', 'w') # format='NETCDF4'
nc_obj.description = 'River sources of Barium concentration in Arctic'
print(nc_obj.file_format)

NETCDF4


In [6]:
Ba_ORCA2_2deg.shape

(12L, 149L, 182L)

In [7]:
time=nc_obj.createDimension('time', None)
lat=nc_obj.createDimension('lat', 149)
lon=nc_obj.createDimension('lon', 182)

In [8]:
print(time.isunlimited())

True


In [9]:
time_counter_obj=nc_obj.createVariable('time_counter', np.float32, ('time',), zlib=True)
BaVar_obj=nc_obj.createVariable('Ba_ORCA2_2deg', np.float64, ('time', 'lat', 'lon'), zlib=True)
d18OVar_obj=nc_obj.createVariable('d18O_ORCA2_2deg', np.float64, ('time', 'lat', 'lon'), zlib=True)
nav_lat_obj=nc_obj.createVariable('nav_lat', np.float64, ('lat', 'lon'), zlib=True)
nav_lon_obj=nc_obj.createVariable('nav_lon', np.float64, ('lat', 'lon'), zlib=True)

In [10]:
time_counter_obj[:]=time_counter
BaVar_obj[:]=Ba_ORCA2_2deg
d18OVar_obj[:]=d18O_ORCA2_2deg
nav_lat_obj[:]=nav_lat
nav_lon_obj[:]=nav_lon

In [11]:
nc_obj.close()

## Initial field

In [22]:
MAT = scipy.io.loadmat('_data/Exchange/NEMO_ORCA2_Ba.mat')
Ba_orca = MAT['Ba_ini_orca'][:]
MAT = scipy.io.loadmat('_data/Exchange/NEMO_ORCA2_d18O.mat')
d18O_orca = MAT['d18O_ini_orca'][:]
#Ba_orca[np.isnan(Ba_orca)] = -999
d18O_name=glob.glob('_data/Exchange/NEMO_ORCA2_d18O_Grid.mat')
d18O_Mat=scipy.io.loadmat(d18O_name[0])
d18O_grid=d18O_Mat['d18O_grid_orca']

boundary_name=glob.glob('_data/Exchange/TRC_BOUND.mat')
Boundary_Mat=scipy.io.loadmat(boundary_name[0])
Ba_boundary=Boundary_Mat['Ba']
d18O_boundary=Boundary_Mat['d18O']

In [23]:
ini_obj = nc.Dataset('_data/Exchange/TRC_ORCA2_Nomask.nc', 'w') # format='NETCDF4'
#ini_obj.description = 'Barium Initial Field'
print(ini_obj.file_format)

NETCDF4


In [24]:
time=ini_obj.createDimension('time', None)
lev = ini_obj.createDimension('lev', 31)
lat=ini_obj.createDimension('lat', 149)
lon=ini_obj.createDimension('lon', 182)

In [25]:
time_counter_obj=ini_obj.createVariable('time_counter', np.float32, ('time',), zlib=True)
BaVar_obj=ini_obj.createVariable('Ba', np.float64, ('time', 'lev', 'lat', 'lon'), zlib=True)
d18OVar_obj=ini_obj.createVariable('d18O', np.float64, ('time', 'lev', 'lat', 'lon'), zlib=True)
d18OGVar_obj=ini_obj.createVariable('d18O_grid', np.float64, ('time', 'lev', 'lat', 'lon'), zlib=True)
BaBVar_obj=ini_obj.createVariable('Ba_boundary', np.float64, ('time', 'lev', 'lat', 'lon'), zlib=True)
d18OBVar_obj=ini_obj.createVariable('d18O_boundary', np.float64, ('time', 'lev', 'lat', 'lon'), zlib=True)
nav_lat_obj=ini_obj.createVariable('nav_lat', np.float64, ('lat', 'lon'), zlib=True)
nav_lon_obj=ini_obj.createVariable('nav_lon', np.float64, ('lat', 'lon'), zlib=True)

In [26]:
#
Ba = np.empty([1, 31, 149, 182])
Ba[0, :, :, :] = Ba_orca

BaB = np.empty([1, 31, 149, 182])
BaB[0, :, :, :] = Ba_boundary
#
d18O = np.empty([1, 31, 149, 182])
d18O[0, :, :, :] = d18O_orca
#
d18OB = np.empty([1, 31, 149, 182])
d18OB[0, :, :, :] = d18O_boundary
#
d18OG = np.empty([1, 31, 149, 182])
d18OG[0, :, :, :] = d18O_grid

In [28]:
time_counter_obj[:]=3600.0
BaVar_obj[:]=Ba
BaBVar_obj[:]=BaB
d18OVar_obj[:]=d18O
d18OBVar_obj[:]=d18OB
d18OGVar_obj[:]=d18OG
nav_lat_obj[:]=nav_lat
nav_lon_obj[:]=nav_lon

In [29]:
BaVar_obj.units='1e-6 mol/L'

In [30]:
ini_obj.close()

In [31]:
test = nc.Dataset('_data/Exchange/TRC_ORCA2_Nomask.nc', 'r')
test.variables['d18O_grid'][:]

array([[[[ -1.76163746e-01,  -2.14759873e-01,  -2.46982081e-01, ...,
           -1.35750703e-01,  -1.76163746e-01,  -2.14759873e-01],
         [ -1.89849455e-01,  -2.28455119e-01,  -2.60744057e-01, ...,
           -1.49986463e-01,  -1.89849455e-01,  -2.28455119e-01],
         [ -2.03344488e-01,  -2.42031498e-01,  -2.74340701e-01, ...,
           -1.64191549e-01,  -2.03344488e-01,  -2.42031498e-01],
         ..., 
         [ -4.29559609e+00,  -4.28632447e+00,  -4.26729748e+00, ...,
           -4.29875022e+00,  -4.29559609e+00,  -4.28632447e+00],
         [ -4.28143008e+00,  -4.28622486e+00,  -4.28143008e+00, ...,
           -4.27910049e+00,  -4.28143008e+00,  -4.28622486e+00],
         [ -4.26729748e+00,  -4.28632447e+00,  -4.29559609e+00, ...,
           -4.25876368e+00,  -4.26729748e+00,  -4.28632447e+00]],

        [[ -2.75193543e-01,  -3.05802724e-01,  -3.28132236e-01, ...,
           -2.37990932e-01,  -2.75193543e-01,  -3.05802724e-01],
         [ -2.81366565e-01,  -3.11867533e-01,

# ANHA4

## River source & boundary

In [ ]:
Ba_name=glob.glob('_data/Exchange/Ba_ANHA4_2deg.mat')
Ba_Mat=scipy.io.loadmat(Ba_name[0])
Ba_ANHA4_2deg=Ba_Mat['Ba_ANHA4']
nav_lon=Ba_Mat['nav_lon']
nav_lat=Ba_Mat['nav_lat']

In [ ]:
boundary_name=glob.glob('_data/Exchange/Ba_boundary_ANHA4.mat')
Boundary_Mat=scipy.io.loadmat(boundary_name[0])
Ba_boundary=Boundary_Mat['Ba_boundary']

In [ ]:
nc_obj = nc.Dataset('_data/Exchange/Ba_ANHA4_SRC.nc', 'w') # format='NETCDF4'
nc_obj.description = 'River sources of Barium concentration in Arctic'
print(nc_obj.file_format)

In [ ]:
time=nc_obj.createDimension('time', None)
lat=nc_obj.createDimension('lat', 149)
lon=nc_obj.createDimension('lon', 182)

In [ ]:
time_counter_obj=nc_obj.createVariable('time_counter', np.float32, ('time',), zlib=True)
BaVar_obj=nc_obj.createVariable('Ba_ANHA4', np.float64, ('time', 'lat', 'lon'), zlib=True)
BaB_obj = nc_obj.createVariable('Ba_boundary', np.float64, ('lat', 'lon'), zlib=True)
nav_lat_obj=nc_obj.createVariable('nav_lat', np.float64, ('lat', 'lon'), zlib=True)
nav_lon_obj=nc_obj.createVariable('nav_lon', np.float64, ('lat', 'lon'), zlib=True)

In [ ]:
time_counter_obj[:]=time_counter
BaVar_obj[:]=Ba_ANHA4
BaB_obj[:]=Ba_boundary
nav_lat_obj[:]=nav_lat
nav_lon_obj[:]=nav_lon

## Initial field

In [ ]:
MAT = scipy.io.loadmat('_data/Exchange/NEMO_ANHA4_Ba.mat')
Ba_ANHA4 = MAT['Ba_ini_ANHA4'][:]

In [ ]:
ini_obj = nc.Dataset('_data/Exchange/Ba_ANHA4_Nomask.nc', 'w') # format='NETCDF4'
ini_obj.description = 'Barium Initial Field'
print(ini_obj.file_format)

In [ ]:
time=ini_obj.createDimension('time', None)
lev = ini_obj.createDimension('lev', 31)
lat=ini_obj.createDimension('lat', 149)
lon=ini_obj.createDimension('lon', 182)

In [ ]:
time_counter_obj=ini_obj.createVariable('time_counter', np.float32, ('time',), zlib=True)
BaVar_obj=ini_obj.createVariable('Ba', np.float64, ('time', 'lev', 'lat', 'lon'), zlib=True)
nav_lat_obj=ini_obj.createVariable('nav_lat', np.float64, ('lat', 'lon'), zlib=True)
nav_lon_obj=ini_obj.createVariable('nav_lon', np.float64, ('lat', 'lon'), zlib=True)

In [ ]:
Ba = np.empty([1, 31, 149, 182])
Ba[0, :, :, :] = Ba_ANHA4

In [ ]:
time_counter_obj[:]=3600.0
BaVar_obj[:]=Ba
nav_lat_obj[:]=nav_lat
nav_lon_obj[:]=nav_lon

In [ ]:
ini_obj.close()